# Bioinformatica

## Задание по программированию: Дифференциально экспрессированные гены

### Описание используемых данных

Данные для этой задачи взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор  генов, которые позволили  бы более точно диагностировать возникновение рака груди на самых ранних стадиях.

В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).

Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей. 

Секвенирование — это определение степени активности генов в анализируемом образце с помощью подсчёта количества соответствующей каждому гену РНК.

В данных для этого задания вы найдете именно эту количественную меру активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.

Вам нужно будет определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

Кроме того, вам нужно будет оценить не только статистическую, но и практическую значимость этих результатов, которая часто используется в подобных исследованиях.

Диагноз человека содержится в столбце под названием "Diagnosis".

### Практическая значимость изменения

Цель исследований — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого часто используется метрика, которая называется fold change (кратность изменения). Определяется она следующим образом:

### Инструкции к решению задачи

Задание состоит из трёх частей. Если не сказано обратное, то уровень значимости нужно принять равным 0.05.

### Часть 1: применение t-критерия Стьюдента

В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

   1. для групп normal (control) и early neoplasia (treatment)
   2. для групп early neoplasia (control) и cancer (treatment)

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости. 

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.stats.multitest as smm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

In [5]:
print(np.__version__)
print(pd.__version__)
print(sc.__version__)
print(statsmodels.__version__)

1.19.2
1.2.1
1.5.2
0.12.1


In [6]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [7]:
data = pd.read_csv('gene_high_throughput_sequencing.csv', sep=',')
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [8]:
data.Diagnosis.unique()

array(['normal', 'early neoplasia', 'cancer'], dtype=object)

In [9]:
data['Diagnosis'].value_counts()

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

In [10]:
data.shape

(72, 15750)

In [11]:
normal = data[data['Diagnosis'] == 'normal'].drop(['Patient_id','Diagnosis'], axis=1)
cancer = data[data['Diagnosis'] == 'cancer'].drop(['Patient_id','Diagnosis'], axis=1)
neoplasia = data[data['Diagnosis'] == 'early neoplasia'].drop(['Patient_id','Diagnosis'], axis=1)

In [12]:
print(normal.shape)
print(cancer.shape)
print(neoplasia.shape)

(24, 15748)
(23, 15748)
(25, 15748)


In [13]:
# scipy.stats.ttest_ind(data.Placebo, data.Methylphenidate, equal_var = False)

In [14]:
genes = normal.columns.tolist()

In [15]:
%time

normal_vs_neoplasia = []

for gene in genes:
    p = sc.stats.ttest_ind(normal[gene], neoplasia[gene], equal_var = False).pvalue
    normal_vs_neoplasia.append([gene, p])

CPU times: user 8 µs, sys: 2 µs, total: 10 µs
Wall time: 17.2 µs


In [16]:
normal_vs_neoplasia_df = pd.DataFrame.from_records(normal_vs_neoplasia)
normal_vs_neoplasia_df.columns = ['Gene', 'normal_vs_neoplasia_p_value']

In [17]:
normal_vs_neoplasia_df

,Gene,normal_vs_neoplasia_p_value
0,LOC643837,0.690766
1,LOC100130417,0.000032
2,SAMD11,0.060273
3,NOC2L,0.826429
4,KLHL17,0.049876
...,...,...
15743,DDX3Y,0.640904
15744,CD24,0.830134
15745,CYorf15B,0.670395
15746,KDM5D,0.793925


In [18]:
%time

neoplasia_vs_cancer = []

for gene in genes:
    p = sc.stats.ttest_ind(neoplasia[gene], cancer[gene], equal_var= False).pvalue
    neoplasia_vs_cancer.append([gene, p])

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs


In [19]:
neoplasia_vs_cancer_df = pd.DataFrame.from_records(neoplasia_vs_cancer)
neoplasia_vs_cancer_df.columns = ['Gene', 'neoplasia_vs_cancer_p_value']

In [20]:
neoplasia_vs_cancer_df

,Gene,neoplasia_vs_cancer_p_value
0,LOC643837,0.413735
1,LOC100130417,0.653429
2,SAMD11,0.079556
3,NOC2L,0.287581
4,KLHL17,0.463292
...,...,...
15743,DDX3Y,0.659369
15744,CD24,0.330617
15745,CYorf15B,0.542939
15746,KDM5D,0.565753


In [21]:
pvalue_df = normal_vs_neoplasia_df.merge(neoplasia_vs_cancer_df)
pvalue_df.index = pvalue_df['Gene']
del pvalue_df['Gene']
pvalue_df

,normal_vs_neoplasia_p_value,neoplasia_vs_cancer_p_value
Gene,,
LOC643837,0.690766,0.413735
LOC100130417,0.000032,0.653429
SAMD11,0.060273,0.079556
NOC2L,0.826429,0.287581
KLHL17,0.049876,0.463292
...,...,...
DDX3Y,0.640904,0.659369
CD24,0.830134,0.330617
CYorf15B,0.670395,0.542939


In [22]:
pvalue_df[pvalue_df['normal_vs_neoplasia_p_value'] < 0.05].shape

(1575, 2)

In [23]:
pvalue_df[pvalue_df['neoplasia_vs_cancer_p_value'] < 0.05].shape

(3490, 2)

In [25]:
with open('bioinformatic_1.1.txt','w') as fout:
    fout.write(str(pvalue_df[pvalue_df['normal_vs_neoplasia_p_value'] < 0.05].shape[0]))

In [26]:
with open('bioinformatic_1.2.txt','w') as fout:
    fout.write(str(pvalue_df[pvalue_df['neoplasia_vs_cancer_p_value'] < 0.05].shape[0]))

### Часть 2: поправка методом Холма

Для этой части задания вам понадобится модуль multitest из statsmodels.

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.51.51.5.

Обратите внимание, что

   - применять поправку на множественную проверку  нужно  ко всем  значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия.  
   - при использовании поправки на уровне значимости 0.025 меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом 0.025, а не 0.05)!


In [28]:
pvalue_df['normal_mean_expression'] = normal.mean()
pvalue_df['neoplasia_mean_expression'] = neoplasia.mean()
pvalue_df['cancer_mean_expression'] = cancer.mean()

In [30]:
def abs_fold_change(c, t):
    if t > c:
        return t/c
    else:
        return c/t
    

In [44]:
pvalue_df['normal_vs_neoplasia_fold_change'] = list(map(abs_fold_change, pvalue_df.normal_mean_expression, pvalue_df.neoplasia_mean_expression))
pvalue_df['neoplasia_vs_cancer_fold_change'] = list(map(abs_fold_change, pvalue_df.neoplasia_mean_expression, pvalue_df.cancer_mean_expression))

In [60]:
pvalue_df['normal_vs_neoplasia_rej_hb'] = smm.multipletests(pvalue_df.normal_vs_neoplasia_p_value, alpha=0.025, method='h')[0]
pvalue_df['neoplasia_vs_cancer_rej_hb'] = smm.multipletests(pvalue_df.neoplasia_vs_cancer_p_value, alpha=0.025, method='h')[0]

In [61]:
pvalue_df

,normal_vs_neoplasia_p_value,neoplasia_vs_cancer_p_value,normal_mean_expression,neoplasia_mean_expression,cancer_mean_expression,normal_vs_neoplasia_fold_change,neoplasia_vs_cancer_fold_change,normal_vs_neoplasia_rej_hb,neoplasia_vs_cancer_rej_hb,normal_vs_neoplasia_rej_bh,neoplasia_vs_cancer_rej_bh
Gene,,,,,,,,,,,
LOC643837,0.690766,0.413735,2.681277,2.510894,2.186060,1.067858,1.148593,False,False,False,False
LOC100130417,0.000032,0.653429,4.368497,8.721781,8.190456,1.996517,1.064871,False,False,False,False
SAMD11,0.060273,0.079556,15.159566,18.531325,23.692614,1.222418,1.278517,False,False,False,False
NOC2L,0.826429,0.287581,15.374351,15.071854,16.468034,1.020070,1.092635,False,False,False,False
KLHL17,0.049876,0.463292,21.459886,24.152469,25.035813,1.125471,1.036574,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
DDX3Y,0.640904,0.659369,1.701654,1.961449,2.292908,1.152672,1.168987,False,False,False,False
CD24,0.830134,0.330617,17.106405,17.567902,15.752268,1.026978,1.115262,False,False,False,False
CYorf15B,0.670395,0.542939,1.576004,1.760707,2.157294,1.117197,1.225243,False,False,False,False


In [62]:
pvalue_df[(pvalue_df.normal_vs_neoplasia_rej_hb) & (pvalue_df.normal_vs_neoplasia_fold_change > 1.5)].shape

(2, 11)

In [63]:
pvalue_df[(pvalue_df.neoplasia_vs_cancer_rej_hb) & (pvalue_df.neoplasia_vs_cancer_fold_change > 1.5)].shape

(77, 11)

In [49]:
with open('bioinformatic_2.1.txt','w') as fout:
    fout.write(str(pvalue_df[(pvalue_df.normal_vs_neoplasia_rej_hb) & (pvalue_df.normal_vs_neoplasia_fold_change > 1.5)].shape[0]))

In [50]:
with open('bioinformatic_2.2.txt','w') as fout:
    fout.write(str(pvalue_df[(pvalue_df.neoplasia_vs_cancer_rej_hb) & (pvalue_df.neoplasia_vs_cancer_fold_change > 1.5)].shape[0]))

### Часть 3: поправка методом Бенджамини-Хохберга

Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.Для этой части задания вам понадобится модуль multitest из statsmodels.

 Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от H0H_0H0​, когда они есть, и будут чаще отклонять H0H_0H0​, когда отличий нет).

В качестве ответа к этому заданию требуется  ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5. 

In [52]:
pvalue_df['normal_vs_neoplasia_rej_bh'] = smm.multipletests(pvalue_df.normal_vs_neoplasia_p_value, alpha=0.025, method='fdr_i')[0]
pvalue_df['neoplasia_vs_cancer_rej_bh'] = smm.multipletests(pvalue_df.neoplasia_vs_cancer_p_value, alpha=0.025, method='fdr_i')[0]

In [54]:
pvalue_df.normal_vs_neoplasia_rej_bh.value_counts()

False    15744
True         4
Name: normal_vs_neoplasia_rej_bh, dtype: int64

In [56]:
pvalue_df[(pvalue_df.normal_vs_neoplasia_rej_bh) & (pvalue_df.normal_vs_neoplasia_fold_change > 1.5)].shape

(4, 11)

In [57]:
pvalue_df[(pvalue_df.neoplasia_vs_cancer_rej_bh) & (pvalue_df.neoplasia_vs_cancer_fold_change > 1.5)].shape

(524, 11)

In [58]:
with open('bioinformatic_3.1.txt','w') as fout:
    fout.write(str(pvalue_df[(pvalue_df.normal_vs_neoplasia_rej_bh) & (pvalue_df.normal_vs_neoplasia_fold_change > 1.5)].shape[0]))

In [59]:
with open('bioinformatic_3.2.txt','w') as fout:
    fout.write(str(pvalue_df[(pvalue_df.neoplasia_vs_cancer_rej_bh) & (pvalue_df.neoplasia_vs_cancer_fold_change > 1.5)].shape[0]))